In [1]:
"""Deploy SRCNN Accellerator."""
import numpy as np
import pynq

# load bitstream
overlay = pynq.Overlay('./srcnn.bit')
overlay.download()

# get srcnn IP reference
srcnn = overlay.srcnn_0
srcnn.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input_ftmap_1 = Register(input_ftmap=write-only),
  input_ftmap_2 = Register(input_ftmap=write-only),
  conv1_weights_1 = Register(conv1_weights=write-only),
  conv1_weights_2 = Register(conv1_weights=write-only),
  conv1_biases_1 = Register(conv1_biases=write-only),
  conv1_biases_2 = Register(conv1_biases=write-only),
  conv1_output_ftmap_1 = Register(conv1_output_ftmap=write-only),
  conv1_output_ftmap_2 = Register(conv1_output_ftmap=write-only),
  conv2_weights_1 = Register(conv2_weights=write-only),
  conv2_weights_2 = Register(conv2_weights=write-only),
  conv2_biases_1 = Register(conv2_biases=write-only),
  conv2_biases_2 = Register(conv2_biases=write-only),

In [2]:
def normalise(pixel):
    return np.single(pixel/255)

In [4]:
from pynq import allocate

H = 255
W = 255
N0 = 1
N1 = 64
N2 = 32
N3 = 1
F1 = 9
F2 = 1
F3 = 5


#allocate buffers
input_image =  allocate((N0, H, W), np.int16)
conv1_w =  allocate((N1, N0, F1, F1), np.int16)
conv1_b =  allocate((N1), np.int8)
conv1_output =  allocate((N1, H, W), np.int16)
conv2_w = allocate((N2, N1, F2, F2), np.int16)
conv2_b = allocate((N2), np.int8)
conv2_output =  allocate((N2, H, W), np.int16)
conv3_w = allocate((N3, N2, F3, F3), np.int16)
conv3_b = allocate((N3), np.int8)
output_image =  allocate((N3, H, W), np.int16)

# input_image =  allocate((N0, H, W), np.uint16)
# conv1_w =  allocate((N1, N0, F1, F1), np.uint16)
# conv1_b =  allocate((N1), np.uint16)
# conv1_output =  allocate((N1, H, W), np.uint32)
# conv2_w = allocate((N2, N1, F2, F2), np.uint32)
# conv2_b = allocate((N2), np.uint16)
# conv2_output =  allocate((N2, H, W), np.uint64)
# conv3_w = allocate((N3, N2, F3, F3), np.uint32)
# conv3_b = allocate((N3), np.uint16)
# output_image =  allocate((N3, H, W), np.uint32)

#instruct IP the DRAM memroy location of allocated vectors
srcnn.write(srcnn.register_map.input_ftmap_1.address, input_image.physical_address)
srcnn.write(srcnn.register_map.conv1_weights_1.address, conv1_w.physical_address)
srcnn.write(srcnn.register_map.conv1_biases_1.address, conv1_b.physical_address)
srcnn.write(srcnn.register_map.conv1_output_ftmap_1.address, conv1_output.physical_address)
srcnn.write(srcnn.register_map.conv2_weights_1.address, conv2_w.physical_address)
srcnn.write(srcnn.register_map.conv2_biases_1.address, conv2_b.physical_address)
srcnn.write(srcnn.register_map.conv2_output_ftmap_1.address, conv2_output.physical_address)
srcnn.write(srcnn.register_map.conv3_weights_1.address, conv3_w.physical_address)
srcnn.write(srcnn.register_map.conv3_biases_1.address, conv3_b.physical_address)
srcnn.write(srcnn.register_map.output_ftmap_1.address, output_image.physical_address)

# convert to fixed point
# def convert(a, total_bits, frac_bits): 
#     condition = 1 << (total_bits - 1) 
#     mask = (~((1 << total_bits) - 1)) & 0xFFFFFFFF 
#     return np.where(a < condition, a, (a.view('u4') | mask).view('i4')) / (1 << frac_bits)


# conv1_w[:] = np.ndarray((1024,), 'i4' dtype='uint16')
# float_ar = np.arange(-512, 512, dtype='f4')
# fixed_ar[:] = float_ar * 256







#load data
conv1_w[:] = np.array(np.fromfile(open("./weights/conv1_weights_3x_flp.bin", "r"), dtype=np.single)).reshape(N1, N0, F1, F1)
conv1_b[:] = np.array(np.fromfile(open("./weights/conv1_biases_3x_flp.bin", "r"), dtype=np.single)).reshape(N1)
conv2_w[:] = np.array(np.fromfile(open("./weights/conv2_weights_3x_flp.bin", "r"), dtype=np.single)).reshape(N2, N1, F2, F2)
conv2_b[:] = np.array(np.fromfile(open("./weights/conv2_biases_3x_flp.bin", "r"), dtype=np.single)).reshape(N2)
conv3_w[:] = np.array(np.fromfile(open("./weights/conv3_weights_3x_flp.bin", "r"), dtype=np.single)).reshape(N3, N2, F3, F3)
conv3_b[:] = np.array(np.fromfile(open("./weights/conv3_biases_3x_flp.bin", "r"), dtype=np.single)).reshape(N3)

def load_input(filename):
    input_image[:] = np.array([normalise(x) for x in np.fromfile(open(filename, "r"), dtype=np.uint8)]).reshape(N0,H,W)



In [5]:
#sanity check
load_input("./tests/set5/butterfly_3x_LR_u8.bin")

In [ ]:
%%timeit -r 1 -n 1

ctrl = srcnn.register_map.CTRL
ctrl.AP_START = 1
progress = 0
while ctrl.AP_DONE != 1:
    if progress%500000 == 0:
        print("still running")
    progress += 1
    pass


still running
still running
still running
still running
still running
still running
still running
still running
still running
still running
still running
still running


In [ ]:
golden_output = np.fromfile(open("./tests/set5/butterfly_3x_GR_flp.bin", "r"), dtype=np.single).reshape(N3,H,W)
mse = np.square(np.subtract(golden_output, output_image)).mean()
print(mse)

In [7]:
import time

def test_srcnn(filename):
    file_input_prefix = "./tests/set14/inputs/"
    file_output_prefix = "./tests/set14/outputs/"
    load_input(file_input_prefix + filename)
    output_file = file_output_prefix + filename[:-9] + "GT_u8.bin"
    tic = time.perf_counter()
    ctrl = srcnn.register_map.CTRL
    ctrl.AP_START = 1
    while ctrl.AP_DONE != 1:
        pass
    toc = time.perf_counter()
    print(filename, "image complete: Elapsed time: ", toc - tic, " seconds")
    golden_output = np.array([normalise(x) for x in np.fromfile(open(output_file, "r"), dtype=np.uint8)]).reshape(N3,H,W)
    mse = np.square(np.subtract(golden_output, output_image)).mean()
    print("Mean squared error: ", mse)
    return (toc - tic, mse)
    

In [8]:
from os import listdir
from os.path import isfile, join


inputs = [f for f in listdir("./tests/set14/inputs") if isfile(join("./tests/set14/inputs", f))]
ave_mse = 0
fps = 0

for i in inputs:
    result = test_srcnn(i)
    ave_mse += result[1]
    fps += result[0]

ave_mse /= len(inputs)
fps /= len(inputs)

print("Average MSE: ", ave_mse)
print("FPS: ", fps)
                             

KeyboardInterrupt: 